# Utilities

1. Web scrape the matchups for each week

## 1.   Web scrape the matchups for each week

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from dotenv import load_dotenv
import os

In [7]:
# Load environment variables
load_dotenv()
cookies_string = os.getenv('COOKIES_STRING')
url_week_1 = os.getenv('URL_WEEK_1')

Python-dotenv could not parse statement starting at line 4


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def scrape_games_selenium(url):
    driver = None
    try:
        options = webdriver.ChromeOptions()
        # Run in visible mode for debugging
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--window-size=1920,1080')
        
        print("Initializing Chrome driver...")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        
        print("Loading page...")
        driver.get(url)
        
        # Wait for initial load
        wait = WebDriverWait(driver, 20)
        
        # Debug: Print the entire page source after initial load
        print("\nChecking page source length...")
        print(f"Page source length: {len(driver.page_source)}")
        
        # Wait and scroll multiple times to ensure content loads
        print("\nScrolling and waiting for content...")
        for i in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(2)
        
        # Try different selectors
        selectors_to_try = [
            ('css', '.ConfidenceProposition'),
            ('css', '.Outcome'),
            ('css', '.OutcomeDetails-title'),
            ('xpath', "//div[contains(@class, 'ConfidenceProposition')]"),
            ('xpath', "//div[contains(@style, 'display: block;')]"),
            ('xpath', "//div[contains(@class, 'Outcome')]")
        ]
        
        for selector_type, selector in selectors_to_try:
            print(f"\nTrying selector: {selector_type} -> {selector}")
            try:
                if selector_type == 'css':
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                else:
                    elements = driver.find_elements(By.XPATH, selector)
                print(f"Found {len(elements)} elements")
                
                if elements:
                    print("Sample element text:", elements[0].text if elements[0].text else "No text")
                    print("Sample element HTML:", elements[0].get_attribute('outerHTML'))
            except Exception as e:
                print(f"Error with selector: {e}")
        
        # Try to find games within an iframe
        print("\nChecking for iframes...")
        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        print(f"Found {len(iframes)} iframes")
        
        if iframes:
            for i, iframe in enumerate(iframes):
                try:
                    print(f"\nSwitching to iframe {i}")
                    driver.switch_to.frame(iframe)
                    
                    # Try to find game elements in iframe
                    elements = driver.find_elements(By.CSS_SELECTOR, '.ConfidenceProposition')
                    print(f"Found {len(elements)} game elements in iframe")
                    
                    driver.switch_to.default_content()
                except Exception as e:
                    print(f"Error with iframe {i}: {e}")
                    driver.switch_to.default_content()
        
        print("\nChecking for shadow DOM elements...")
        shadow_hosts = driver.find_elements(By.CSS_SELECTOR, "[class*='confidence']")
        print(f"Found {len(shadow_hosts)} potential shadow hosts")
        
        # If we still haven't found the elements, check the full DOM structure
        print("\nAnalyzing DOM structure...")
        dom_analysis = driver.execute_script("""
            function analyzeDOM(element, depth = 0) {
                let result = '';
                if (element.className) {
                    result += '  '.repeat(depth) + element.tagName + ' - Classes: ' + element.className + '\\n';
                }
                for (let child of element.children) {
                    result += analyzeDOM(child, depth + 1);
                }
                return result;
            }
            return analyzeDOM(document.body);
        """)
        print("DOM Structure (first 1000 chars):")
        print(dom_analysis[:1000])
        
        return None
        
    except Exception as e:
        print(f"Error scraping: {e}")
        return None
        
    finally:
        if driver:
            try:
                driver.quit()
            except:
                pass

print("Starting scraping process...")
url = "https://fantasy.espn.com/games/college-football-pickem-2024/picks?id=da1a1900-6505-11ef-9300-e73470a18359"
games_df = scrape_games_selenium(url)

Starting scraping process...
Initializing Chrome driver...
Loading page...

Checking page source length...
Page source length: 610281

Scrolling and waiting for content...

Trying selector: css -> .ConfidenceProposition
Found 0 elements

Trying selector: css -> .Outcome
Found 0 elements

Trying selector: css -> .OutcomeDetails-title
Found 0 elements

Trying selector: xpath -> //div[contains(@class, 'ConfidenceProposition')]
Found 0 elements

Trying selector: xpath -> //div[contains(@style, 'display: block;')]
Found 0 elements

Trying selector: xpath -> //div[contains(@class, 'Outcome')]
Found 0 elements

Checking for iframes...
Found 12 iframes

Switching to iframe 0
Found 0 game elements in iframe

Switching to iframe 1
Found 0 game elements in iframe

Switching to iframe 2
Found 0 game elements in iframe

Switching to iframe 3
Found 0 game elements in iframe

Switching to iframe 4
Found 0 game elements in iframe

Switching to iframe 5
Found 0 game elements in iframe

Switching to ifr